https://www.superdatascience.com/pages/deep-learning -> course documents
This is actually a movie created by a RNN. Additional reading: https://arstechnica.com/gaming/2016/06/an-ai-wrote-this-movie-and-its-strangely-moving/

Reference: http://colah.github.io/posts/2015-08-Understanding-LSTMs/

The goal is to predict the trend of the Google Stock price

In [1]:
# Importing the Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Settings to display more with print
pd.options.display.max_rows = 200
pd.set_option('display.max_columns', 999)

In [5]:
# Part 1 - Data Preprocessing (Steps 1-3)
# The RNN will only be trained on the training set. It will not see the test set until after the training is done!
dataset_train = pd.read_csv('dataset/Google_Stock_Price_Train.csv')
training_set = dataset_train.iloc[:,1:2].values # creates a data frame (numpy array)
# getting the Open prices of the Google stock price
# first : refers to all rows
    # Do not want just a vector array. Want a 2d array for np. This means we must import an array for the number of columns.
    # The trick is to reference 1:2 because in python the upperbound is excluded. so only column index[1] is taken but imported
    # in the correct format (numpy array of 1 column)
    # the .values makes it a np array
print(type(dataset_train))
print(type(training_set))

# Feature Scaling

#MinMaxScaler normalizes the data
sc = MinMaxScaler(feature_range = (0,1), copy=True) #sc refers to scale
#fit means that it will get the min and max and will scale the data. The transform will compute the scaled (normalized) value based on the normalization formula.
training_set_scaled = sc.fit_transform(training_set)

<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [6]:
print(training_set_scaled)

[[0.08581368]
 [0.09701243]
 [0.09433366]
 ...
 [0.95725128]
 [0.93796041]
 [0.93688146]]


In [7]:
len(training_set_scaled)

1258

In [8]:
# Step 4

# Creating a data structure with 60 timesteps and 1 output
# the 60 timestep means that the model will look at time t, the previous 60 times steps and time step t. Based on those values
# the model will try to predict the next output. 
# The value of 60 timesteps was chosen by trial and error. There are ~20 buisness days in a month so 60 timesteps is 60 buisness days (3 months)
X_train = [] #input (60 previous days)
y_train = [] #output (stock price for the next financial day)
#these are lists

for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0]) #upperbound is excluded
    y_train.append(training_set_scaled[i, 0])

# y_train is the array that needs to be predicted based on the data in X_train
# converting lists to np arrays
X_train, y_train = np.array(X_train), np.array(y_train)

In [9]:
print(X_train)
print(X_train.shape)
print("------------------")
print("Break for viewing")
print("------------------")
print(y_train)
print(y_train.shape)

[[0.08581368 0.09701243 0.09433366 ... 0.07846566 0.08034452 0.08497656]
 [0.09701243 0.09433366 0.09156187 ... 0.08034452 0.08497656 0.08627874]
 [0.09433366 0.09156187 0.07984225 ... 0.08497656 0.08627874 0.08471612]
 ...
 [0.92106928 0.92438053 0.93048218 ... 0.95475854 0.95204256 0.95163331]
 [0.92438053 0.93048218 0.9299055  ... 0.95204256 0.95163331 0.95725128]
 [0.93048218 0.9299055  0.93113327 ... 0.95163331 0.95725128 0.93796041]]
(1198, 60)
------------------
Break for viewing
------------------
[0.08627874 0.08471612 0.07454052 ... 0.95725128 0.93796041 0.93688146]
(1198,)


In [10]:
print(X_train.shape[0])
print(X_train.shape[1])

1198
60


In [22]:
# Step 5

# Reshaping
# 1st argument is the np.array you want to reshape
# the 2nd argument is the new shape we want the array to have
# the 3rd dimension is the new indicator or number of indicators (new input dimenions). 
# want 3 dimensions for X_train with the 3rd corresponding to the indicator (prediction)
# Information for the recurrent input shapes: https://keras.io/layers/recurrent/
# Look under the headers "Input shapes"
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    # 1 corresponds to the Google Open Stock Price

In [23]:
print(X_train)
print(X_train.shape)
print("------------------")
print("Break for viewing")
print("------------------")
print(y_train)
print(y_train.shape)

[[[0.08581368]
  [0.09701243]
  [0.09433366]
  ...
  [0.07846566]
  [0.08034452]
  [0.08497656]]

 [[0.09701243]
  [0.09433366]
  [0.09156187]
  ...
  [0.08034452]
  [0.08497656]
  [0.08627874]]

 [[0.09433366]
  [0.09156187]
  [0.07984225]
  ...
  [0.08497656]
  [0.08627874]
  [0.08471612]]

 ...

 [[0.92106928]
  [0.92438053]
  [0.93048218]
  ...
  [0.95475854]
  [0.95204256]
  [0.95163331]]

 [[0.92438053]
  [0.93048218]
  [0.9299055 ]
  ...
  [0.95204256]
  [0.95163331]
  [0.95725128]]

 [[0.93048218]
  [0.9299055 ]
  [0.93113327]
  ...
  [0.95163331]
  [0.95725128]
  [0.93796041]]]
(1198, 60, 1)
------------------
Break for viewing
------------------
[0.08627874 0.08471612 0.07454052 ... 0.95725128 0.93796041 0.93688146]
(1198,)


# Part 2 - Building the RNN
    # Building a LSTM
    # Look up CTC

In [3]:
# Step 6
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Using TensorFlow backend.
/Users/ryanwinfree/opt/anaconda3/envs/udemy4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ryanwinfree/opt/anaconda3/envs/udemy4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ryanwinfree/opt/anaconda3/envs/udemy4/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.in

In [4]:
import keras
print(keras.__version__)
#print(Tensorflow.__version__)

2.1.2


In [12]:
# Step 7
# Initialising the RNN
regressor = Sequential()
    # represents a sequence of layers

# LSTM class
# Adding the first LSTM layer and some Dropout regularisation (added to avoid overfitting). Don't want overfitting in general
# LSTM first layer. Need to use the LSTM class. Need to input 3 arguments
    # number of units = number of LSTM memory cells to have in the layer. Like neurons
    # 2nd arg. return sequences. Set to true because building a stacked LSTM. So have to return values if having multiple LSTM layers
    # 3rd arg. input shape. The shape of the input (e.g. shape of X_train which is 3D). Only have to specify 2 of the 3 dimensions though
        # X_train.shape[1] is the time steps
        # 1 corresponds to the indicators (predictors)
# Only have to specify the input shape for the first layer
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1) ))

# Adding dropout regularisation
    # standard to dropout 20%. So 20% of the neurons will be ignored in the backward and forward propegation 
    # (so 10 neurons)
    # these are ignored each iteration of the training
regressor.add(Dropout(0.2))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [13]:
# Step 8
# Adding a 2nd LSTM layer and some dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

In [14]:
# Step 8
# Adding a 3rd LSTM layer and some dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))
# Adding a 4th LSTM layer and some dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = False))
# setting return_sequences because we don't return it on the last LSTM layer? 
regressor.add(Dropout(0.2))

In [15]:
# Step 9; Adding the output layer
# Adding a fully connected layer to the last LSTM layer. Use the Dense class
# 1 neuron for the output layer (1 in this case).
regressor.add(Dense(units=1))

In [17]:
# Step 10
# Compiling the RNN. 
# Input the optimizer (look at the keras documentation: keras.io). 
# RMSprop is usually a good optimizer to use with RNNs. However, 
# using a different one due to experimentation 
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [24]:
print(X_train)
print(X_train.shape)
print("------------------")
print("Break for viewing")
print("------------------")
print(y_train)
print(y_train.shape)

[[[0.08581368]
  [0.09701243]
  [0.09433366]
  ...
  [0.07846566]
  [0.08034452]
  [0.08497656]]

 [[0.09701243]
  [0.09433366]
  [0.09156187]
  ...
  [0.08034452]
  [0.08497656]
  [0.08627874]]

 [[0.09433366]
  [0.09156187]
  [0.07984225]
  ...
  [0.08497656]
  [0.08627874]
  [0.08471612]]

 ...

 [[0.92106928]
  [0.92438053]
  [0.93048218]
  ...
  [0.95475854]
  [0.95204256]
  [0.95163331]]

 [[0.92438053]
  [0.93048218]
  [0.9299055 ]
  ...
  [0.95204256]
  [0.95163331]
  [0.95725128]]

 [[0.93048218]
  [0.9299055 ]
  [0.93113327]
  ...
  [0.95163331]
  [0.95725128]
  [0.93796041]]]
(1198, 60, 1)
------------------
Break for viewing
------------------
[0.08627874 0.08471612 0.07454052 ... 0.95725128 0.93796041 0.93688146]
(1198,)


In [ ]:
# Step 11 (final step of part 2): Fitting the data to training set
# Fitting the RNN to the Training Set
# y_train = the "ground truth", aka the answers for the RNN
# batch size = the batches of stock prices going into the NN at once. 
regressor.fit(X_train, y_train, epochs = 105, batch_size = 32)

Epoch 1/105
1198/1198 [==============================] - 23s 19ms/step - loss: 0.0460
Epoch 2/105
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0066
Epoch 3/105
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0056
Epoch 4/105
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0054
Epoch 5/105
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0046
Epoch 6/105
1198/1198 [==============================] - 17s 15ms/step - loss: 0.0044
Epoch 7/105
1198/1198 [==============================] - 18s 15ms/step - loss: 0.0046
Epoch 8/105
1198/1198 [==============================] - 20s 17ms/step - loss: 0.0037
Epoch 9/105
1198/1198 [==============================] - 19s 16ms/step - loss: 0.0046
Epoch 10/105
1198/1198 [==============================] - 17s 14ms/step - loss: 0.0045
Epoch 11/105
1198/1198 [==============================] - 17s 14ms/step - loss: 0.0039
Epoch 12/105
1198/1198 [============================

# Part 3 - Making the predictions and visulising the results